<a href="https://colab.research.google.com/github/gandalf1819/20Newsgroups-data-mining/blob/master/20Newsgroup_data_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirrors.viethosting.com/apache/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")

In [0]:
import os
import sys
import pyspark
import string
import csv
import json
import statistics
from itertools import combinations
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import types as D
from pyspark.sql.window import Window

In [0]:
!pip install lime

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pyspark
from pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql import functions as F

from pyspark.sql import types as D
from pyspark.ml.feature import HashingTF, IDF, RegexTokenizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
import csv
import lime 
from lime import lime_text
from lime.lime_text import LimeTextExplainer

import numpy as np

In [0]:
sc = SparkContext()

ValueError: ignored

In [0]:
spark = SparkSession \
        .builder \
        .appName("hw3") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [0]:
# Load data
categories = ["alt.atheism", "soc.religion.christian"]
LabeledDocument = pyspark.sql.Row("category", "text")

In [0]:
def categoryFromPath(path):
    return path.split("/")[-2]

In [0]:
def prepareDF(typ):
    rdds = [sc.wholeTextFiles("/content/gdrive/My Drive/BigData-hw3/20news-bydate/20news-bydate-" + typ + "/" + category)\
              .map(lambda x: LabeledDocument(categoryFromPath(x[0]), x[1]))\
            for category in categories]
    return sc.union(rdds).toDF()

In [0]:
train_df = prepareDF("train").cache()
test_df  = prepareDF("test").cache()

Task 1.1

In [0]:
# Append text to file
def writeToFile(text):
	with open('/content/gdrive/My Drive/BigData-hw3/cnw282_report.txt', 'a', newline="") as f:
		f.write(text)

In [0]:
train_df_rows = 'Number of rows in train_df = ' + str(train_df.count())+'\n'
test_df_rows = 'Number of rows in test_df = ' + str(test_df.count())+'\n'

In [0]:
# Dump the results to the report
writeToFile("Task 1.1 (a)\n")
writeToFile(train_df_rows)
writeToFile(test_df_rows)

In [0]:
train_df = train_df.rdd.zipWithIndex()
train_df = train_df.toDF()
train_df = train_df.withColumn('category', train_df['_1'].getItem("category"))
train_df = train_df.withColumn('text', train_df['_1'].getItem("text"))
train_df = train_df.withColumnRenamed('_2', 'id')
train_df = train_df.select('id','category','text')

In [0]:
test_df = test_df.rdd.zipWithIndex()
test_df = test_df.toDF()
test_df = test_df.withColumn('category', test_df['_1'].getItem("category"))
test_df = test_df.withColumn('text', test_df['_1'].getItem("text"))
test_df = test_df.withColumnRenamed('_2', 'id')
test_df = test_df.select('id','category','text')

In [0]:
writeToFile("\nTask 1.1 (b)\n")
writeToFile("First 5 rows of 'INDEXED' test set \n\n")
k = test_df.take(5)
for i,row in enumerate(k):
	row_name='Row-' + str(i)
	writeToFile(row_name+'\n')
	writeToFile(str(row[0])+', '+str(row[1])+ ', '+str(row[2])+ '\n\n')

In [0]:
# Build pipeline and run
indexer   = StringIndexer(inputCol="category", outputCol="label")
tokenizer = RegexTokenizer(pattern=u'\W+', inputCol="text", outputCol="words", toLowercase=False)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
idf       = IDF(inputCol="rawFeatures", outputCol="features")
lr        = LogisticRegression(maxIter=20, regParam=0.001)

# Builing model pipeline
pipeline = Pipeline(stages=[indexer, tokenizer, hashingTF, idf, lr])

# Train model on training set
model = pipeline.fit(train_df)   #if you give new names to your indexed datasets, make sure to make adjustments here

# Model prediction on test set
pred = model.transform(test_df)  # ...and here

# Model prediction accuracy (F1-score)
pl = pred.select("label", "prediction").rdd.cache()
metrics = MulticlassMetrics(pl)
init_f1_score = metrics.fMeasure()
print('f1 score: ',init_f1_score)

f1 score:  0.9483960948396095


In [0]:
# Dump f1 score to report
writeToFile("\nTask 1.2 (a)\n")
writeToFile("F1 score: ")
writeToFile(str(init_f1_score))

pred.show()

# Dump schema to report
writeToFile("\nTask 1.2 (b)\n")
for tup in pred.dtypes:
	res='Column Name: ' + str(tup[0])+', '+ 'Type: ' + str(tup[1]) + '\n'
	writeToFile(res)

+---+-----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|   category|                text|label|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+---+-----------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0|alt.atheism|From: sandvik@new...|  1.0|[From, sandvik, n...|(262144,[1838,365...|(262144,[1838,365...|[-9.6765787643934...|[6.27318353598026...|       1.0|
|  1|alt.atheism|From: kempmp@phoe...|  1.0|[From, kempmp, ph...|(262144,[14,619,1...|(262144,[14,619,1...|[1.91251820260104...|[0.87130179057815...|       0.0|
|  2|alt.atheism|From: rsrodger@wa...|  1.0|[From, rsrodger, ...|(262144,[3030,330...|(262144,[3030,330...|[-1.5061726689323...|[0.18150669621399...|       1.0|
|  3|alt.atheism|From: I3150101@db

Task 1.2

In [0]:
#Use LIME to explain example
class_names = ['Atheism', 'Christian']
explainer = LimeTextExplainer(class_names=class_names)

# Choose a random text in test set, change seed for randomness 
test_point = test_df.sample(False, 0.1, seed = 10).limit(1)
test_point_label = test_point.select("category").collect()[0][0]
test_point_text = test_point.select("text").collect()[0][0]

In [0]:
def classifier_fn(data):
    spark_object = spark.createDataFrame(data, "string").toDF("text")
    pred = model.transform(spark_object)   #if you build the model with a different name, make appropriate changes here
    output = np.array((pred.select("probability").collect())).reshape(len(data),2)
    return output

In [0]:
exp = explainer.explain_instance(test_point_text, classifier_fn, num_features=6)

/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]


Task 1.3<br>
Task 1.4

In [0]:
test_IDS = pred.filter((F.col('id')==0)|(F.col('id')==275)|(F.col('id')==664)).collect()

misclassified = pred.filter((F.col('label')!=F.col('prediction'))).collect()

misclassified_list=[]

In [0]:
for row in misclassified:
	Id = row[0]
	prob_l = row[8]
	conf = abs(prob_l[0]-prob_l[1])
	exp_row = explainer.explain_instance(row[2], classifier_fn, num_features=6)
	new_row = [Id, conf, exp_row.as_list()]
	misclassified_list.append(new_row)

misclassified_list.sort(key= lambda k: (k[1], k[0]), reverse=True)

header=['ID', 'Confidence', 'Explanation-List']

/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a

In [0]:
# Task 1.4
with open('/content/gdrive/My Drive/BigData-hw3/cnw282_misclassified_ordered.csv', "w", newline="") as f:
	writer = csv.writer(f)
	writer.writerow(i for i in header)
	writer.writerows(misclassified_list)

task_ID=[]
for row in test_IDS:
	Id=row[0]
	category=row[1]
	prob_list=row[8]
	pred_category='atheism' if row[9]==1.0 else 'christian'
	exp_row=explainer.explain_instance(row[2], classifier_fn, num_features=6)
	task_ID.append([Id, category, pred_category, prob_list, exp_row.as_list()])

writeToFile("\nTask 1.3\n")
for row in task_ID:
	Id='Id: '+ str(row[0])
	label='Actual Category: ' + str(row[1])
	pred_label='Predicted Category: ' + str(row[2])
	prob='Probability: ' + str(row[3])
	explanation='Explanation List: ' + str(row[4])
	line=Id +', '+label+', '+ pred_label+ ', '+prob+ ', '+ explanation+ '\n'
	writeToFile(line)

/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]
/usr/local/lib/python3.6/dist-packages/lime/lime_text.py:116: FutureWarning: split() requires a non-empty pattern match.
  self.as_list = [s for s in splitter.split(self.raw) if s]


Task 1.5

In [0]:
words = {}

for row in misclassified_list:
	exp_list = row[2]
	for tup in exp_list:
		word = tup[0]
		weight = tup[1]
		if word not in words.keys():
			words[word]=[]
			words[word].append(1)
			words[word].append(abs(weight))
		elif word in words.keys():
			words[word][0] = words[word][0]+1
			words[word][1] = words[word][1]+abs(weight)

words_arr=[]

words_new_arr=[]

# For report
for k, v in words.items():
	new_row = [k, v[0], v[1]]
	words_arr.append(new_row)

# For Task 2
for k, v in words.items():
	n_row = [k, v[1]/v[0]]
	words_new_arr.append(n_row)

words_new_arr.sort(key=lambda k: (k[1]), reverse=True)	

words_arr.sort(key=lambda k: (k[1], k[2]), reverse=True)
header1 = ['Word', 'Count', 'Weight']
	
# Task 1.5
with open('/content/gdrive/My Drive/BigData-hw3/cnw282_words_weight.csv', 'w', newline="") as f1:
	writer = csv.writer(f1)
	writer.writerow(i for i in header1)
	writer.writerows(words_arr)

Task 2

In [0]:
t5 = [row[0] for row in words_new_arr[:20]]					
remove_word = F.udf(lambda x: x.replace(t5[0], "").replace(t5[1], "").replace(t5[2], "").replace(t5[3], "").replace(t5[4],"").replace(t5[5],"").replace(t5[6],"").replace(t5[7],"").replace(t5[8],"").replace(t5[9],"").replace(t5[10], "").replace(t5[11], "").replace(t5[12],"").replace(t5[13],"")  , D.StringType())
train_df = train_df.withColumn('text1', remove_word(train_df.text))
train_df = train_df.select('id', 'category', 'text1')
train_df = train_df.withColumnRenamed("text1", "text")
train_df.show(2)
model = pipeline.fit(train_df)
pred = model.transform(test_df)
pl = pred.select("label", "prediction").rdd.cache()
metrics = MulticlassMetrics(pl)
print('new f1 score = ', metrics.fMeasure())
new_misclassified = pred.filter((F.col('label')!=F.col('prediction'))).collect()
new_misclassified_IDs = [row[0] for row in new_misclassified]
prev_missclassified_IDs = [row[0] for row in misclassified_list]
correct_ids_after=[]

for Id in prev_missclassified_IDs:
	if Id not in new_misclassified_IDs:
		correct_ids_after.append(Id)

print("Final: ", correct_ids_after)

writeToFile("\n\nTask 2\n")
writeToFile("\nStrategy:\n")
writeToFile("Step-1:\n") 
writeToFile("We have identified the words which contributed towards misclassified documents. Using weights/count as measure, we will remove the misclassified words which contributed to a decrease in accuracy of the model. Along with this, we have the words in a sorted order in descending order starting with words whcih misclassified the most number of documents.\n") 
writeToFile("We can put the top 13 words in a list that had contributed the most towards misclassified document\n")
writeToFile("\nStep-2:\n") 
writeToFile("Create a new column called 'text1' from text with the use of user defined function - udf. Rename this column to 'text' later\n")
writeToFile("\nStep-3:\n")
writeToFile("After making these modifications, we can observe that we get a better accuracy as we have optimized the model\n\n")

writeToFile("We can say that words that contributed to multiple misclassified documents contributed for the decrease of the precision of the model - Without loss of generality (WLOG)\n")
writeToFile("After removing some of these words we reduce our rate of false positive (FPR) and false negatives (FNR) which will contribute to the increase in the F1 score\n\n")

acc = "New Accuracy after Feature Engineering: " + str(metrics.fMeasure())+'\n'
# Dump new accuracy to report
writeToFile(acc)

# Dump ID's that got classified correctly after feature selection
correct_ids = "Document ID's that are classified correctly after feature selection (which were misclassified before): " + str(correct_ids_after)+'\n'
writeToFile(correct_ids)

+---+-----------+--------------------+
| id|   category|                text|
+---+-----------+--------------------+
|  0|alt.atheism|From: kv07@IASTAT...|
|  1|alt.atheism|From: mas@Cadence...|
+---+-----------+--------------------+
only showing top 2 rows

new f1 score =  0.9595536959553695
Final:  [165, 121, 288, 81, 160, 271, 259, 258, 275]
